WASTE MANAGEMENT
PROBLEM

Waste management is a big problem in our country. Most of the wastes end up in landfills. This leads to many issues like

Increase in landfills
Eutrophication
Consumption of toxic waste by animals
Increase in toxins
Land, water and air pollution
APPROACH

Analysed the components of household waste
Segregated into two classes (Organic and recyclable)
Automated the process by using IOT and machine learning
Reduce toxic waste ending in landfills

In [1]:
# !nvidia-smi

In [2]:
import keras
import tensorflow as tf

2022-01-04 01:08:26.992340: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-04 01:08:26.992406: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [4]:
# from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())

In [5]:
### IMPORT LIBRARIES

In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import skimage.io
import tensorflow 
import tqdm
import glob

from tqdm import tqdm 

from skimage.io import imread, imshow
from skimage.transform import resize

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, BatchNormalization, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.preprocessing.image import load_img, img_to_array

%matplotlib inline

In [9]:
### IMPORT DATASET

In [10]:
train_o = glob.glob('Waste_DATASET/TRAIN/O/*.jpg')
a = len(train_o)

In [11]:
train_r = glob.glob('Waste_DATASET/TRAIN/R/*.jpg')
b = len(train_r)

In [12]:
# Total training images 
print("Nos of training samples: {}".format(a+b))

Nos of training samples: 22564


In [13]:
#### DATA AUGMENTATION

In [14]:
train_datagen = ImageDataGenerator(rescale = 1.0 / 255.0,
                                   zoom_range = 0.4,
                                   rotation_range = 10,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   validation_split = 0.2)

valid_datagen = ImageDataGenerator(rescale = 1.0 / 255.0,
                                   validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1.0 / 255.0)

In [15]:
# train_dataset  = train_datagen.flow_from_directory(directory = 'Waste_DATASET/TRAIN',
#                                                    target_size = (224,224),
#                                                    class_mode = 'binary',
#                                                    batch_size = 128, 
#                                                    subset = 'training')

train_dataset  = train_datagen.flow_from_directory(directory = 'Waste_DATASET/TRAIN',
                                                   target_size = (224,224),
                                                   class_mode = 'binary',
                                                   batch_size = 64, 
                                                   subset = 'training')

Found 18052 images belonging to 2 classes.


In [16]:
# valid_dataset = valid_datagen.flow_from_directory(directory = 'Waste_DATASET/TRAIN',
#                                                   target_size = (224,224),
#                                                   class_mode = 'binary',
#                                                   batch_size = 128, 
#                                                   subset = 'validation')

valid_dataset = valid_datagen.flow_from_directory(directory = 'Waste_DATASET/TRAIN',
                                                  target_size = (224,224),
                                                  class_mode = 'binary',
                                                  batch_size = 64, 
                                                  subset = 'validation')

Found 4512 images belonging to 2 classes.


In [17]:
# Class Indices 

train_dataset.class_indices

{'O': 0, 'R': 1}

In [18]:
# # Viewing Images

# # fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20,20))

# for i in tqdm(range(12)):
#     rand1 = np.random.randint(len(train_dataset))
#     rand2 = np.random.randint(128)
#     ax[i].imshow(train_dataset[rand1][0][rand2])
#     ax[i].axis('off')
#     label = train_dataset[rand1][1][rand2]
#     if label == 1:
#         ax[i].set_title('Recycle Waste')
#     else:
#         ax[i].set_title('Organic Waste')

In [19]:
### MODEL BUILDING

In [20]:
# Defining Model

base_model = VGG16(input_shape=(224,224,3), 
                   include_top=False,
                   weights="imagenet")

2021-10-23 16:48:29.399424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-23 16:48:29.399477: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-23 16:48:29.399514: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (4ca66b5771c3): /proc/driver/nvidia/version does not exist
2021-10-23 16:48:29.399773: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
# Freezing Layers 

for layer in base_model.layers:
    layer.trainable=False

In [22]:
# Summary

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [23]:
# Defining Layers

model=Sequential()
model.add(base_model) # VGG16 

## change 
model.add(Dropout(0.2))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
## change 


model.add(Dense(1,activation='sigmoid'))  # output layer 

In [24]:
# Summary

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 25088)             100352    
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
activation (Activation)      (None, 1024)              0

In [25]:
# Model Compile 

OPT    = tensorflow.keras.optimizers.Adam(learning_rate=0.001)  # Adam => GSD, # change learning_rate 0.01 ==> runs faster 

model.compile(loss='binary_crossentropy',
              metrics=[tensorflow.keras.metrics.AUC(name = 'auc')],
              optimizer=OPT)

In [26]:
# Defining Callbacks

filepath = './best_weights.hdf5'

earlystopping = EarlyStopping(monitor = 'val_auc', 
                              mode = 'max' , 
                              patience = 5,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_auc', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [27]:
# Model Fitting 

model_history=model.fit(train_dataset,
                        validation_data=valid_dataset,
                        epochs = 10,    # change num of epochs 
                        callbacks = callback_list,
                        verbose = 1)

2021-10-23 16:48:39.656959: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2021-10-23 16:48:42.390391: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 822083584 exceeds 10% of free system memory.


2021-10-23 16:48:42.591942: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 822083584 exceeds 10% of free system memory.


  1/283 [..............................] - ETA: 35:40 - loss: 0.7333 - auc: 0.5992

...

 98/283 [=========>....................] - ETA: 14:47 - loss: 0.4005 - auc: 0.9117

 99/283 [=========>....................] - ETA: 14:42 - loss: 0.4005 - auc: 0.9116

100/283 [=========>....................] - ETA: 14:37 - loss: 0.4000 - auc: 0.9117

101/283 [=========>....................] - ETA: 14:32 - loss: 0.4004 - auc: 0.9116

102/283 [=========>....................] - ETA: 14:28 - loss: 0.4012 - auc: 0.9113

In [0]:
### MODEL EVALUATION

In [0]:
# Summarize the model loss

plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('VGG16 Model Loss', FontSize = 16)
plt.ylabel('Loss', FontSize = 16)
plt.xlabel('Epoch', FontSize = 16)
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [0]:
# Summarize models auc

plt.plot(model_history.history['auc'])
plt.plot(model_history.history['val_auc'])
plt.title('VGG16 Model Accuracy', FontSize = 16)
plt.ylabel('Accuracy',FontSize = 16)
plt.xlabel('Epoch',FontSize = 16)
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [0]:
max(model_history.history['val_auc']), max(model_history.history['val_auc'])

In [0]:
# Test Data 

test_data = test_datagen.flow_from_directory(directory = 'Waste_DATASET/TEST',
                                             target_size = (224,224),
                                             class_mode = 'binary',
                                             batch_size = 128)

In [0]:
# Evaluating Loss and AUC - Test Data 

model.evaluate(test_data)

In [0]:
# Test Case:1 - ORGANIC

dic = test_data.class_indices
idc = {k:v for v,k in dic.items()}

img = load_img('Waste_DATASET/TEST/O/O_12650.jpg', target_size=(224,224))
img = img_to_array(img)
img = img / 255
imshow(img)
plt.axis('off')
img = np.expand_dims(img,axis=0)
answer = model.predict_proba(img)

if answer[0][0] > 0.5:
    print("The image belongs to Recycle waste category")
else:
    print("The image belongs to Organic waste category ")

In [0]:
# Test Case:2 - RECYCLE

dic = test_data.class_indices
idc = {k:v for v,k in dic.items()}

img = load_img('Waste_DATASET/TEST/R/R_10011.jpg', target_size=(224,224))
img = img_to_array(img)
img = img / 255
imshow(img)
plt.axis('off')
img = np.expand_dims(img,axis=0)
answer = model.predict_proba(img)

if answer[0][0] > 0.5:
    print("The image belongs to Recycle waste category")
else:
    print("The image belongs to Organic waste category ")

In [0]:
# CONCLUSION

# 1. We were able to classify images properly having accuracy of 97.00% in training dataset.

# 2. We acheived an accuracy of 95.60% on validation data and 94.98% accuracy on test accuracy.